In [ ]:
# import duckdb
import polars as pl
from operator import itemgetter
from ipyleaflet import Map, Marker, AwesomeIcon
from ipywidgets import Layout, HTML

In [ ]:
pl.Config.set_fmt_str_lengths(900)

In [ ]:
%reload_ext sql

In [ ]:
%sql duckdb:////data/duckdb/foursquare.duckdb

In [ ]:
%%sql
INSTALL httpfs;
LOAD httpfs;
INSTALL spatial;
LOAD spatial;

In [ ]:
%%sql
SELECT COUNT(*) AS links
  FROM fs
 WHERE website IS NOT NULL


In [ ]:
%%sql
SELECT * from categories 
WHERE fsq_category_label LIKE '%istor%'

In [ ]:
home_location = (51.69220544124235, -2.222896814346314)
# home_location = (55.32102, -1.89418)
home_lat, home_lon = home_location

bikes = "4bf58dd8d48988d115951735"
music = "4bf58dd8d48988d1fe941735"
venue = "4bf58dd8d48988d1f2931735"
sites = "4deefb944765f83613cdba6e"

In [ ]:
%%sql
DROP TABLE IF EXISTS subset

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS subset AS 
 FROM gb 
WHERE contains(fsq_category_ids, '{{sites}}')

In [ ]:
%%sql 
pike_df << SELECT name, website, address, locality, region, postcode, country, 
                  latitude, longitude, point, 
                  ST_Distance(point, ST_Point({{home_lat}}, {{home_lon}})) AS distance
             FROM subset
            WHERE ST_DWithin(point, ST_Point({{home_lat}}, {{home_lon}}), 100)
            ORDER BY distance ASC

In [ ]:
print(pike_df)

In [ ]:
map = Map(
    center=home_location,
    zoom=15,
    layout=Layout(width="100%", height="800px"),
)

home_icon = AwesomeIcon(
    name="home",
    marker_color="orange",
    icon_color="white",
    spin=False,
)

home = Marker(
    location=home_location,
    draggable=False,
    color="red",
    icon=home_icon,
    title="Pike Lane Cottage",
    z_index_offset=1000,
)

map.add(home)


get_addr = itemgetter(
    "address",
    "locality",
    "region",
    "postcode",
    "country",
)

empty_icon = AwesomeIcon(
    name="question",
    marker_color="green",
    icon_color="white",
    spin=False,
)

link_icon = AwesomeIcon(
    name="link",
    marker_color="blue",
    icon_color="white",
    spin=False,
)

for place in pike_df.head(500).to_dicts():
    name = place["name"]
    marker_icon = empty_icon
    z_index = 0
    if place["website"]:
        name = f'<a href="{place["website"]}">{name}</a>'
        marker_icon = link_icon
        z_index = 100

    marker = Marker(
        location=(place["latitude"], place["longitude"]),
        title=place["name"],
        draggable=False,
        icon=marker_icon,
        z_index_offset=z_index,
    )

    address_parts = [part for part in get_addr(place) if part is not None]
    address = "<br>".join(address_parts)

    popup = HTML()
    popup.value = f"<h3>{name}</h3><div>{address}</div>"
    marker.popup = popup
    map.add(marker)


display(map)